In [ ]:
# Customer Segmentation Dashboard: Phase 3 - Calculating RFM Scores

**Objective**: Compute RFM (Recency, Frequency, Monetary) metrics for each customer and store the results for segmentation.

**Steps**:
1. Load cleaned data from SQLite.
2. Set reference date (day after last transaction).
3. Calculate Recency, Frequency, and Monetary values for each customer.
4. Store RFM values in a new table and CSV.

In [1]:
import pandas as pd
import sqlite3

# Connect to SQLite database
db_path = '../ecommerce_data.db'
conn = sqlite3.connect(db_path)

# Load cleaned data
df = pd.read_sql_query('SELECT * FROM transactions_clean', conn)
print('Loaded cleaned data shape:', df.shape)
df.head()

Loaded cleaned data shape: (401604, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSpend
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [3]:
# Ensure InvoiceDate is datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Set reference date (day after last transaction)
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print('Reference date for recency calculation:', reference_date)

Reference date for recency calculation: 2011-12-10 12:50:00


In [4]:
# Calculate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalSpend': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
print(rfm.head())

   CustomerID  Recency  Frequency  Monetary
0     12346.0      326          2      0.00
1     12347.0        2          7   4310.00
2     12348.0       75          4   1797.24
3     12349.0       19          1   1757.55
4     12350.0      310          1    334.40


In [5]:
# Save RFM table to SQLite
rfm.to_sql('rfm', conn, if_exists='replace', index=False)

# Save to CSV
rfm.to_csv('../data/rfm.csv', index=False)
print('RFM table saved to SQLite and CSV.')

RFM table saved to SQLite and CSV.
